# Day 3 — UNet Slicers & Partial Real-Weights Load

This notebook mirrors the Day-3 smoke test you ran from the CLI, but keeps a reproducible, runnable record here.

**Goals**
1. Verify cached shapes from Day-2 (video/audio latents).
2. Instantiate UNet slicers for SVD & AudioLDM2.
3. Load *real ALDM2 UNet weights* (with config patch) and confirm shapes flow.

**Key script:** `scripts/smoke/day03_slicers_smoke.py` (called via `make smoke-day03`).

In [ ]:
# Env check + switch to repo rootimport os, sys, subprocess, json, pathlib, platformimport torchROOT = "/workspace/jointdit"os.chdir(ROOT)print("CWD:", os.getcwd())print("Python:", sys.version.split(" ")[0], "on", platform.platform())print("Torch:", torch.__version__, "CUDA:", torch.cuda.is_available())if torch.cuda.is_available():    print("GPU:", torch.cuda.get_device_name(0))

## Optional: Mirror real ALDM2 UNet weights locally
- If you want to re-download the **real** `cvssp/audioldm2` UNet to the assets folder, set `DO_MIRROR=True` below.
- If you've already mirrored & edited `cross_attention_dim` in the JSON (to a scalar, e.g. `1024`), leave this as-is.

In [ ]:
DO_MIRROR = False  # set True to (re)mirrorif DO_MIRROR:    from diffusers import UNet2DConditionModel    mdl_id = "cvssp/audioldm2"    print("Downloading UNet from:", mdl_id)    unet = UNet2DConditionModel.from_pretrained(mdl_id, subfolder="unet")    out = f"{ROOT}/assets/models/audioldm2/unet"    pathlib.Path(out).mkdir(parents=True, exist_ok=True)    unet.save_pretrained(out)    print("Saved to:", out)else:    print("Skipping mirror; using paths in configs/day03_models.yaml")

## Run the Day-3 slicer smoke
This executes `make smoke-day03`, which loads SVD & AudioLDM2 UNets (with config patch), prints param counts, and passes shape-through checks.

In [ ]:
import os, subprocessos.chdir(ROOT)env = os.environ.copy()env["PYTHONPATH"] = "."proc = subprocess.run(["bash","-lc","make smoke-day03"], capture_output=True, text=True, env=env)print(proc.stdout)print(proc.stderr)print("Return code:", proc.returncode)assert proc.returncode == 0, "Smoke failed — check stderr above."

### Expected signals
- Lines like:
  - `[ALDM2][realweights] loaded 688/688 tensors | mismatched: 832 | missing: 0`
  - `[SVD] input→(T, C, H, W) → ... (same)` and `[ALDM2] input→... (same)`
- Final line: `[OK] Day-3 slicer scaffold is healthy (shapes preserved, params present).`

If you see config errors about `cross_attention_dim` being a list, fix by editing the mirrored UNet config JSON at:
`/workspace/jointdit/assets/models/audioldm2/unet/config.json` → set `"cross_attention_dim": 1024` (or 768) as a single integer.